In [3]:
import pandas as pd
import os

# 1. 병합 대상 팀원 이름 리스트
team_members = ['장윤서', '노현비', '김정운', '강지수', '황영재']

# 2. 현재 디렉토리에서 해당 파일들 병합
merged_df = pd.DataFrame()
for member in team_members:
    file_name = f'{member}.csv'
    if os.path.exists(file_name):
        try:
            temp_df = pd.read_csv(file_name, encoding='CP949')  # 한글 인코딩 처리
        except UnicodeDecodeError:
            temp_df = pd.read_csv(file_name, encoding='utf-8')  # utf-8로 재시도
        temp_df['member'] = member
        merged_df = pd.concat([merged_df, temp_df], ignore_index=True)
    else:
        print(f"⚠️ 파일이 존재하지 않음: {file_name}")

# 3. 결과 저장
merged_df.to_csv('통합.csv', index=False, encoding='utf-8-sig')  # Excel에서 한글 안깨지게 저장
print("✅ 병합 완료: 통합.csv")


c:\Temp\ipykernel_9164\1559830437.py:15: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(file_name, encoding='utf-8')  # utf-8로 재시도


✅ 병합 완료: 통합.csv


In [3]:
pip install selenium

  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   --- ------------------------------------ 0.8/9.4 MB 8.5 MB/s eta 0:00:02
   ------------ --------------------------- 2.9/9.4 MB 6.7 MB/s eta 0:00:01
   ---------------- ----------------------- 3.9/9.4 MB 6.0 MB/s eta 0:00:01
   -------------------------- ------------- 6.3/9.4 MB 7.4 MB/s eta 0:00:01
   ----------------------------------- ---- 8.4/9.4 MB 8.0 MB/s eta 0:00:01
   ---------------------------------------- 9.4/9.4 MB 7.6 MB/s eta 0:00:00
Using cached outcome-1.3.0.post0-py2.py3-none-any.whl (10 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.3
    Uninstalling urllib3-2.2.3:
      Successfully uninstalled urllib3-2.2.3
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_ex


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import re
import time

# 크롤링 함수
def extract_availability_and_sold(url):
    available, sold = None, None
    options = Options()
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    # options.add_argument("--headless")  # 필요 시 주석 해제

    driver = webdriver.Chrome(options=options)
    try:
        driver.get(url)
        time.sleep(1.5)

        try:
            # span[1] → 재고
            avail_text = driver.find_element(By.XPATH, '//*[@id="qtyAvailability"]/span[1]').text.strip()
            match_avail = re.search(r"(\d+)", avail_text)
            if match_avail:
                available = int(match_avail.group(1))
            print(f"✅ available: {available} (텍스트: '{avail_text}')")

            # span[2] → 판매
            sold_text = driver.find_element(By.XPATH, '//*[@id="qtyAvailability"]/span[2]').text.strip()
            match_sold = re.search(r"(\d+)", sold_text)
            if match_sold:
                sold = int(match_sold.group(1))
            print(f"✅ sold: {sold} (텍스트: '{sold_text}')")

        except Exception as e:
            print(f"⚠️ 요소 추출 실패: {e}")

    except Exception as e:
        print(f"❌ 페이지 로딩 실패: {e}")
    finally:
        driver.quit()

    return (available, sold)


# =====================

df = pd.read_csv("RawData_drop_duplicates_윤서.csv", low_memory=False)
sample_urls = df['itemWebUrl'].dropna()
sample_idx = sample_urls.index

available_list = []
sold_list = []

for i, url in enumerate(sample_urls):
    print(f"🔍 ({i+1}/5) 크롤링 중: {url}")
    available, sold = extract_availability_and_sold(url)
    available_list.append(available)
    sold_list.append(sold)

# 결과 저장
df.loc[sample_idx, 'available_quantity'] = available_list
df.loc[sample_idx, 'sold_quantity'] = sold_list

df.to_csv("with_sold_data.csv", index=False)
print("크롤링 완료 및 저장됨!")


🔍 (1/5) 크롤링 중: https://www.ebay.com/itm/197434763288?_skw=Jewelry&hash=item2df8076818:g:sJcAAeSwMj5oTOyT&amdata=enc%3AAQAKAAAA4PeG5RIuIyokJHJy903%2F5UaK2NJXQc0MVJtKJvOwyo0dYP1xQW3OzCwJb%2F8%2BFgsegrnqz4cLcdbzACDqSSlS%2Baa%2BRt4rYXNofAZCh2T2CR1phCo9ysLdsitfEJDFtp1sm3OmHNRrVnyEdjKD56CSobfv6GMsy7773Go%2FQpnoBaOBcJ0oiPUHMVvWnOqSPUl0zsxaNl7F8oj9SX0gZIeikRoLL84hZ4qTYi6dAFYwwFCACnt0G3CrjUHlPLEf72R8BriNTuUgqQxPvCc8viqSxYeNUQSzw49Qayknzqnl%2B369
✅ available: None (텍스트: '')
✅ sold: None (텍스트: '')
🔍 (2/5) 크롤링 중: https://www.ebay.com/itm/255380790240?_skw=Jewelry&hash=item3b75e19be0:g:UQEAAOSwlnNiBgQ7&amdata=enc%3AAQAKAAAA4PeG5RIuIyokJHJy903%2F5UZqkTL2P5F70kzrRkx%2BtW8VJr9OzyTipDdROAUPCbeWjXMEi89vYBEUser0eGWJMId3jDrFjLpgRW%2BprYzmH%2F7gg3ocOxOA2Yps%2BoI%2FnjMdt%2F6wEA1Y6ieRNSrVhYvhBTHxm18znRSNW5rpAh1y0I0Ca33wv4psnVROj26ocKjjNl7emwnh9oWbD8IC%2BgoQfZUe0tqLWjbHg6JfJdk9b7cvwL85ITBttYJC9OXpSraI2oPFDOQvY%2FSebnLLWATo9SzoUVhkncv8%2BTu5eDaaGzUi
✅ available: 10 (텍스트: '10개 있음')
✅ sold: 5 (텍스트: '5,465개 판매됨')

KeyboardInterrupt: 

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from concurrent.futures import ThreadPoolExecutor, as_completed
import re
import time

# 🔧 크롤링 함수
def extract_availability_and_sold(url):
    available, sold = None, None
    options = Options()
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    #options.add_argument("--headless")  # 필요시 사용

    driver = webdriver.Chrome(options=options)
    try:
        driver.get(url)
        time.sleep(1.5)

        try:
            avail_text = driver.find_element(By.XPATH, '//*[@id="qtyAvailability"]/span[1]').text.strip()
            match_avail = re.search(r"(\d+)", avail_text)
            if match_avail:
                available = int(match_avail.group(1))

            sold_text = driver.find_element(By.XPATH, '//*[@id="qtyAvailability"]/span[2]').text.strip()
            match_sold = re.search(r"(\d+)", sold_text)
            if match_sold:
                sold = int(match_sold.group(1))

        except Exception as e:
            print(f"⚠️ 요소 추출 실패: {e}")

    except Exception as e:
        print(f"❌ URL 로딩 실패: {e}")
    finally:
        driver.quit()

    return (available, sold)

# =========================
# 🔁 병렬 실행 로직
# =========================

# 📦 CSV 불러오기
df = pd.read_csv("RawData_drop_duplicates_윤서.csv", low_memory=False)
sample_urls = df['itemWebUrl'].dropna()
sample_idx = sample_urls.index

# 🧠 병렬 크롤링 설정
max_workers = 4  # CPU 코어 수에 맞게 조절 (4~8 추천)
results = [None] * len(sample_urls)

print(f"🌐 {len(sample_urls)}개 병렬 크롤링 시작 (동시 {max_workers}개)")

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    future_to_idx = {
        executor.submit(extract_availability_and_sold, url): i
        for i, url in enumerate(sample_urls)
    }

    for i, future in enumerate(as_completed(future_to_idx)):
        idx = future_to_idx[future]
        try:
            results[idx] = future.result()
        except Exception as e:
            print(f"❌ 크롤링 실패 index {idx}: {e}")
            results[idx] = (None, None)

        if (i + 1) % 100 == 0:
            print(f"✅ ({i + 1}/{len(sample_urls)}) 완료")

# 🎯 결과 적용
available_list, sold_list = zip(*results)
df.loc[sample_idx, 'available_quantity'] = available_list
df.loc[sample_idx, 'sold_quantity'] = sold_list

# 💾 저장
df.to_csv("with_sold_data.csv", index=False)
print("✅ 병렬 크롤링 완료 및 저장됨!")
